See which torsions probably have the biggest torsional barriers, do some sort of check that big torsional barriers are crossed...

In [5]:
from simtk import unit
from openmmtools.constants import kB
temperature = 298.15 * unit.kelvin
kT = kB * temperature

def evaluate_torsion(theta, periodicity=1, phase=0, force_constant=1):
    return force_constant * (1 + np.cos(periodicity * theta - phase))

def plot_torsion_component(periodicity, phase, force_constant):
    x = np.linspace(0, 2 * np.pi, 1000)
    y = evaluate_torsion(x, periodicity, phase, force_constant)
    plt.plot(x / np.pi, y)
    return y

def compute_full_torsion(a, b, c, d):
    x = np.linspace(0, 2 * np.pi, 1000)
    terms = forces_per_torsion[(a,b,c,d)]
    u = sum([evaluate_torsion(x, periodicities[i], phases[i], ks[i]) for i in terms])
    return u

def plot_full_torsion(a,b,c,d):
    u = compute_full_torsion(a,b,c,d)
    x = np.linspace(0, 2 * np.pi, len(u))
    plt.plot(x / np.pi,u)
    return u

def plot_torsion_by_index(i):
    return plot_torsion_component(periodicities[i], phases[i], ks[i])

def plot_marginal(U, temperature = 300 * unit.kelvin):
    '''assume U is unit'd (compatible with kT) and has limits [0,2pi] '''
    x = np.linspace(0, 2 * np.pi, len(u))
    
    q = np.exp(-U / kT)
    Z = 2 * np.pi * np.sum(q) / len(q)
    p = q / Z
    plt.plot(x, p)
    return p

In [ ]:
indices = []
periodicities = []
phases = []
ks = []

for i in range(num_torsions):
    a, b, c, d, periodicity, phase, force_constant = f.getTorsionParameters(i)
    indices.append((a,b,c,d))
    periodicities.append(periodicity)
    phases.append(phase)
    ks.append(force_constant)
    
# which quadruples have more than one torsion force acting on them?

forces_per_torsion = dict()
for ind in indices:
    forces_per_torsion[ind] = []

for i,ind in enumerate(indices):
    forces_per_torsion[ind].append(i)
    
modes_per_torsion = dict()
for ind in indices:
    modes_per_torsion[ind] = 0
    
for i,ind in enumerate(indices):
    modes_per_torsion[ind] += periodicities[i]